In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import random
import math
import SimpleITK as sitk
import mahotas as mt
import collageradiomics
import imageio

from PIL import Image
from matplotlib.patches import Rectangle
from scipy import linalg
from skimage.util.shape import view_as_windows
from matplotlib.gridspec import GridSpec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from itertools import product
from skimage.feature.texture import greycomatrix

import os
import configparser

In [ ]:
# Read image.
img_s = sitk.ReadImage('../../sample_data/ImageSlice.png')
img_array = sitk.GetArrayFromImage(img_s)

In [ ]:
# Create window.
svd_radius = 5
patch_window_width = 30
patch_window_height = 30
mask_min_x = 252
mask_min_y = 193
mask_max_x = mask_min_x + patch_window_width
mask_max_y = mask_min_y + patch_window_height

In [ ]:
# Use static initializer for rectangular option.
collage = collageradiomics.Collage.from_rectangle(img_array, mask_min_x, mask_min_y, patch_window_width, patch_window_height, svd_radius, verbose_logging=True)

In [ ]:
# Run CoLlage Algorithm and return feature.
haralick_features = collage.execute()

In [ ]:
# Display patch.
print(collageradiomics.highlight_rectangle_on_image(img_array, mask_min_x, mask_min_y, patch_window_width, patch_window_height).shape)

In [ ]:
# Display gradient
figure, axes = plt.subplots(1, 2, figsize=(15, 15))
collageradiomics.show_colored_image(figure, axes[0], collage.dx)
axes[0].set_title(f'Gx size={collage.dx.shape}')
collageradiomics.show_colored_image(figure, axes[1], collage.dy)
axes[1].set_title(f'Gy size={collage.dy.shape}')

In [ ]:
# Display dominant angles
figure, axes = plt.subplots(1, 2, figsize=(15, 15))
collageradiomics.show_colored_image(figure, axes[0], collage.dominant_angles_array)
axes[0].set_title('Dominant Angles (SVD)')
collageradiomics.show_colored_image(figure, axes[1], collage.dominant_angles_shaped)
axes[1].set_title('Dominant Angles (Color Binned)')

In [ ]:
# Display dominant angles color-binned
figure, axis = plt.subplots(1,1, figsize=(5,5))
collageradiomics.show_colored_image(figure, axis, collage.dominant_angles_shaped)

In [ ]:
# Display haralick
figure, axes = plt.subplots(3, 5, figsize=(15,15))

for row in range(3):
    for col in range(5):
        feature = row*5+col
        axis = axes[row][col]
        axis.set_axis_off()
        if feature>=13:
            continue
        collageradiomics.show_colored_image(figure, axis, haralick_features[:,:,feature])
        axis.set_title(f'Haralick {feature+1}')

In [ ]:
# Display full Haralick images with masked region.
aspect = img_array.shape[1] / img_array.shape[0]
figsize = 15 / aspect
figure, axes = plt.subplots(3, 5, figsize=(figsize * aspect, figsize))

for row in range(3):
    for col in range(5):
        feature = row*5+col
        axis = axes[row][col]
        axis.set_axis_off()
        if feature>=13:
            continue
        collageradiomics.show_colored_image(figure, axis, collage.full_masked_images[feature])
        axis.set_title(f'Haralick {feature+1}')

In [ ]:
# Show preview of larger version of image.
plt.figure(figsize = (figsize * aspect, figsize))
figure = plt.imshow(collage.full_masked_images[0], cmap = plt.cm.jet)
figure.axes.get_xaxis().set_visible(False)
figure.axes.get_yaxis().set_visible(False)
plt.title('Haralick 1 Preview')

In [ ]:
import logging
logger = logging.getLogger('imageio').setLevel(logging.ERROR)

def scale_array_for_image(array_to_scale):
    flat_array = array_to_scale.flatten()
    minimum = float(min(flat_array))
    maximum = float(max(flat_array))
    array_range = maximum - minimum
    array_to_scale = array_to_scale - minimum
    array_to_scale /= array_range
    array_to_scale *= 255
    return array_to_scale

def write_haralick_image(feature):
    haralick = collageradiomics.scale_array_for_image(collage.haralick_features[:,:,feature])
    haralick = haralick.astype('uint8')
    imageio.imwrite(f'../../output_data/haralick-feature-{feature + 1}.png', plt.cm.jet(haralick))
    img = Image.open(f'../../output_data/haralick-feature-{feature + 1}.png')
    img = img.convert('RGBA')
    pixdata = img.load()

    width, height = img.size
    for y in range(height):
        for x in range(width):
            if pixdata[x, y] == (0, 0, 127, 255) or pixdata[x, y] == (127, 0, 0, 255):
                pixdata[x, y] = (255, 255, 255, 0)

    img.save(f'../../output_data/haralick-feature-{feature + 1}.png', 'PNG')
    background = Image.open('../../sample_data/ImageSlice.png')
    foreground = Image.open(f'../../output_data/haralick-feature-{feature + 1}.png')

    background = background.convert('RGBA')
    background.paste(foreground, (collage.mask_min_x, collage.mask_min_y), foreground)
    background.show()
    background.save(f'../../output_data/haralick-feature-{feature + 1}.png')

In [ ]:
# Writes images to output_data directory in the root of the repository and show them.
print('Original Image')
display(Image.open('../../sample_data/ImageSlice.png'))
for feature in range(13):
    write_haralick_image(feature)
    print(f'Haralick Feature {feature + 1}')
    display(Image.open(f'../../output_data/haralick-feature-{feature + 1}.png'))